# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235258028604                   -0.50    8.0
  2   -7.250244504178       -1.82       -1.40    1.0
  3   -7.251274622046       -2.99       -2.08    4.0
  4   -7.251151933597   +   -3.91       -2.02    4.0
  5   -7.251330634090       -3.75       -2.70    2.0
  6   -7.251337997749       -5.13       -3.12    2.0
  7   -7.251338761654       -6.12       -3.94    1.0
  8   -7.251338789527       -7.55       -4.06    4.0
  9   -7.251338797694       -8.09       -4.50    2.0
 10   -7.251338798153       -9.34       -4.70    1.0
 11   -7.251338798685       -9.27       -5.42    3.0
 12   -7.251338798699      -10.84       -5.61    2.0
 13   -7.251338798704      -11.36       -6.10    2.0
 14   -7.251338798704      -12.17       -6.82    2.0
 15   -7.251338798705      -13.41       -7.05    4.0
 16   -7.251338798705      -14.45       -7.97    3.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05376258820232145
[ Info: Arnoldi iteration step 2: normres = 0.4019064444160188
[ Info: Arnoldi iteration step 3: normres = 0.9747236922615365
[ Info: Arnoldi iteration step 4: normres = 0.386336853318012
[ Info: Arnoldi iteration step 5: normres = 0.3344893138553753
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (4.57e-02, 4.21e-02, 2.16e-01, 2.27e-01, 9.83e-02)
[ Info: Arnoldi iteration step 6: normres = 0.28218832861946014
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.27e-02, 2.33e-01, 1.39e-01, 5.42e-02, 1.19e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08457450724185488
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (5.08e-04, 1.68e-02, 5.84e-03, 1.69e-02, 7.91e-02)
[ Info: Arnoldi iteration step 8: normres = 0.07817275086311982
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.68e-0